# Cubed-Sphere SWE Solver: Benchmarks & Verification

This notebook demonstrates the performance and accuracy of the **Cubed-Sphere Shallow Water Equations (SWE) Solver** using the JAX backend.

We will cover:
1.  **Verification**: Comparing JAX results against a reference NumPy implementation.
2.  **Optimization**: Showcasing the speedup from JAX's JIT compilation and `lax.scan`.
3.  **Scalability**: Measuring performance scaling with grid resolution ($N$).

## 1. Setup & Environment

We enforce hot-reloading and local path priority to ensure the notebook uses the latest source code revisions.

In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

In [ ]:
import sys
import os

# Force current parent directory to be 1st in search path
# Assuming notebook is in 'examples/', so '..' is the repo root
repo_root = os.path.abspath('..')
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print(f"Repo Root added to path: {repo_root}")

In [ ]:
# Optional: Install dependencies if running in a fresh Colab environment
# %pip install -e .

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Tuple, Any, Dict
import gc

# --- JAX Configuration ---
# Updated for T4 GPU: Disable Double Precision (x64)
jax.config.update("jax_enable_x64", False)
# jax.config.update("jax_platform_name", "cpu")

print(f"JAX Backend: {jax.devices()[0].platform.upper()}")
print(f"Double Precision: {jax.config.read('jax_enable_x64')}")

## Part 1: Physics Verification (NumPy vs JAX)

**Goal**: Prove that the optimized JAX solver produces identical physical results to the reference NumPy solver.

We will run a short simulation (Case 2, $N=32$) for 10 steps and compare the **Height field ($h$)**.

In [ ]:
from cubed_sphere.solvers.swe_numpy import CubedSphereSWENumpy

def decode_height(state, solver, backend='numpy'):
    """
    Helper to extract Height field (h) from the state vector.
    State structure: [Mass, Velocity_u, Velocity_v]
    Height h = Mass / sqrt_g
    """
    mass = np.array(state[0])
    h_field = np.zeros_like(mass)

    if backend == 'numpy':
        # NumPy solver: solver.faces is dict
        face_names = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
        for i, fname in enumerate(face_names):
            if fname in solver.faces:
                sqrt_g = solver.faces[fname].sqrt_g
                h_field[i] = mass[i] / sqrt_g
    else:
        # JAX solver: solver.grid_metrics.faces is tuple
        # This attribute MUST exist if CubedSphereSWEJax is correctly initialized
        if not hasattr(solver, 'grid_metrics'):
            raise RuntimeError("Solver missing 'grid_metrics'. Ensure package is up to date and kernel restarted.")

        for i in range(6):
            # Pull sqrt_g to host memory as numpy array
            sqrt_g = np.array(solver.grid_metrics.faces[i].sqrt_g)
            h_field[i] = mass[i] / sqrt_g

    return h_field

# Configuration
config = {
    'N': 32,
    'R': 6.37122e6,
    'dt': 30.0, # Reduced for stability in Float32
    'H_avg': 10000.0,
    'gravity': 9.80616,
    'Omega': 7.292e-5,
    'initial_condition': 'case2', # Set IC explicit in config
    'n_vars': 3,
}

NUM_STEPS = 10

print(f"--- Running Verification (N={config['N']}, Steps={NUM_STEPS}) ---")

# 1. Run NumPy Reference
print("Running NumPy Solver...", end=" ")
solver_np = CubedSphereSWENumpy(config)
# Note: logic for IC might vary, relying on auto-init or explicit call
try:
    u0_np = solver_np.get_initial_condition(type='case2')
except TypeError:
    u0_np = solver_np.get_initial_condition()

state_np = u0_np.copy()
t = 0.0
start_np = time.time()
for _ in range(NUM_STEPS):
    state_np = solver_np.step(t, state_np, config['dt'])
    t += config['dt']
print(f"Done ({time.time() - start_np:.2f}s)")

# 2. Run JAX Implementation
print("Running JAX Solver...", end=" ")
solver_jax = CubedSphereSWEJax(config)
u0_jax = jnp.array(u0_np)
state_jax = u0_jax
t = 0.0

# Force JIT compilation first
_ = solver_jax.step(t, state_jax, config['dt'])

# Actual run
start_jax = time.time()
state_jax = u0_jax # Reset state
for _ in range(NUM_STEPS):
    state_jax = solver_jax.step(t, state_jax, config['dt'])
    t += config['dt']
# Block until finished
state_jax_np = np.array(state_jax)
print(f"Done ({time.time() - start_jax:.2f}s)")

# 3. Compare Results
h_np = decode_height(state_np, solver_np, backend='numpy')
h_jax = decode_height(state_jax_np, solver_jax, backend='jax')

diff = np.abs(h_np - h_jax)
max_diff = np.max(diff)

print(f"\nMax Height Difference: {max_diff:.4e} meters")
print(f"Height Range: [{np.min(h_np):.2f}, {np.max(h_np):.2f}] meters")

# Verification Tolerance
# Metal/GPU backends may have different floating point accumulation.
# We accept a small relative error (~1%) or absolute error compatible with grid resolution effects.
passed = np.allclose(h_np, h_jax, rtol=0.01, atol=100.0)

if passed:
     print(f"✅ Verification PASSED: JAX matches NumPy within tolerance (Diff {max_diff:.2f} < 100m/1%).")
else:
     print(f"⚠️ Verification WARNING: Discrepancy detected (Max Diff: {max_diff:.2f}m).")
     print("This may be due to platform differences (e.g., Metal vs CPU) or algorithm variations.")


## Part 2: The Power of JIT & `lax.scan`

**Goal**: Demonstrate the massive speedup obtained by using `jax.lax.scan` to compile the entire time integration loop.

*   **Slow Path**: Python loop calling `step()` iteratively (similar to standard NumPy usage).
*   **Fast Path**: `lax.scan` unrolling the loop on the device (GPU/TPU/CPU).

In [ ]:
def compute_safe_dt(state: Any, solver: Any, config: Dict[str, Any], cfl: float = 0.1) -> Tuple[float, float]:
    """
    Computes a safe time step based on the maximum wave speed and grid resolution.

    Formula:
    $$dt = CFL \times \frac{\Delta x}{|u| + \sqrt{gH}}$$

    Args:
        state (Any): The current state vector array of shape (3, 6, N, N).
        solver (Any): The instantiated CubedSphereSWEJax solver containing grid metrics.
        config (Dict[str, Any]): Configuration dictionary containing 'gravity', 'R', and 'N'.
        cfl (float, optional): The Courant-Friedrichs-Lewy condition number. Defaults to 0.1.

    Returns:
        Tuple[float, float]: A tuple containing:
            - dt_safe (float): The computed safe time step in seconds.
            - lambda_max (float): The maximum characteristic wave speed in m/s.
    """
    mass = state[0]
    u1_cov = state[1]
    u2_cov = state[2]

    metrics = solver.stacked_metrics
    sqrt_g = metrics.sqrt_g
    g_inv = metrics.g_inv

    h = mass / sqrt_g

    u1_con = g_inv[...,0,0]*u1_cov + g_inv[...,0,1]*u2_cov
    u2_con = g_inv[...,1,0]*u1_cov + g_inv[...,1,1]*u2_cov

    u_sq = u1_cov * u1_con + u2_cov * u2_con
    u_mag = jnp.sqrt(u_sq)

    c_wave = jnp.sqrt(config['gravity'] * h)
    lambda_max = jnp.max(u_mag + c_wave)

    R = config['R']
    N = config['N']
    min_dx = (np.pi * R) / (2 * N)

    dt_safe = cfl * (min_dx / lambda_max)

    return float(dt_safe), float(lambda_max)

def benchmark_scan_speedup() -> Tuple[float, float]:
    N = 32
    config = {
        'N': N,
        'R': 6371220.0,
        'H_avg': 10000.0,
        'gravity': 9.80616,
        'Omega': 7.292e-5,
        'initial_condition': 'case2',
        'n_vars': 3,
    }

    print(f"--- Benchmark Init (N={N}) ---")
    temp_solver = CubedSphereSWEJax(config)
    state_init = temp_solver.get_initial_condition()

    dt_safe, v_max = compute_safe_dt(state_init, temp_solver, config, cfl=0.15)
    config['dt'] = dt_safe

    solver = CubedSphereSWEJax(config)

    print(f"Physics Check: V_max_init = {v_max:.2f} m/s")
    print(f"Computed Safe dt = {dt_safe:.4f} s (CFL=0.15)")

    num_steps = 200
    total_sim_time = dt_safe * num_steps
    print(f"Total Simulation Time: {total_sim_time:.2f} s ({num_steps} steps)")
    print(f"Backend: {jax.devices()[0].platform.upper()}")

    # --- Fast Path (lax.scan) ---
    print("\nRunning Fast Path (lax.scan)...")
    print("  Compiling...", end=" ", flush=True)
    t0 = time.time()
    # WARMUP with block_until_ready
    jax.block_until_ready(solver.solve((0, total_sim_time), state_init, callbacks=None))
    print(f"Done ({time.time()-t0:.2f}s)")

    start = time.time()
    # EXECUTION with block_until_ready
    jax.block_until_ready(solver.solve((0, total_sim_time), state_init, callbacks=None))
    flux_t = time.time() - start

    steps_per_sec_fast = num_steps / flux_t
    print(f"  Execution Time: {flux_t:.4f}s")
    print(f"  Throughput:     {steps_per_sec_fast:.1f} steps/s")

    # --- Slow Path (Python loop) ---
    print("\nRunning Slow Path (Python loop)...")
    def dummy_callback(t, s): pass

    print("  Compiling...", end=" ", flush=True)
    jax.block_until_ready(solver.solve((0, total_sim_time), state_init, callbacks=[dummy_callback]))
    print("Done.")

    start = time.time()
    jax.block_until_ready(solver.solve((0, total_sim_time), state_init, callbacks=[dummy_callback]))
    slow_t = time.time() - start

    steps_per_sec_slow = num_steps / slow_t
    print(f"  Execution Time: {slow_t:.4f}s")
    print(f"  Throughput:     {steps_per_sec_slow:.1f} steps/s")

    speedup = steps_per_sec_fast / steps_per_sec_slow
    print(f"\n🚀 JAX/XLA Speedup Factor: {speedup:.2f}x")

    return steps_per_sec_slow, steps_per_sec_fast

slow_rate, fast_rate = benchmark_scan_speedup()

## Part 3: Scalability Benchmark (SWE)

**Goal**: Analyze how the JAX solver scales with increasing grid resolution ($N$).

We will run the solver for resolutions $N \in [32, 64, 96]$ and measure the wall-clock time per step.

In [ ]:
def benchmark_scalability() -> Tuple[list, list]:
    N_values = [32, 64, 96]
    times_per_step = []

    config_base = {
        'R': 6371220.0,
        'H_avg': 10000.0,
        'gravity': 9.80616,
        'Omega': 7.292e-5,
        'initial_condition': 'case2',
        'n_vars': 3,
    }

    num_steps = 100

    print(f"--- Scalability Benchmark ({num_steps} steps) ---")
    print(f"{'N':<5} | {'dt (s)':<10} | {'Grid Pts':<10} | {'Time/Step (ms)':<15} | {'Throughput (TPS)':<15}")
    print("-" * 65)

    for N in N_values:
        config = config_base.copy()
        config['N'] = N

        temp_solver = CubedSphereSWEJax(config)
        state = temp_solver.get_initial_condition()

        dt_safe, v_max = compute_safe_dt(state, temp_solver, config, cfl=0.15)
        config['dt'] = dt_safe

        solver = CubedSphereSWEJax(config)
        total_sim_time = dt_safe * num_steps

        # WARMUP with block_until_ready
        jax.block_until_ready(solver.solve((0, total_sim_time), state, callbacks=None))

        # EXECUTION
        start = time.time()
        jax.block_until_ready(solver.solve((0, total_sim_time), state, callbacks=None))
        duration = time.time() - start

        avg_time = duration / num_steps
        avg_ms = avg_time * 1000
        tps = 1.0 / avg_time

        times_per_step.append(avg_ms)
        grid_points = 6 * N * N

        print(f"{N:<5} | {dt_safe:<10.3f} | {grid_points:<10} | {avg_ms:<15.2f} | {tps:<15.1f}")

        # MEMORY MANAGEMENT: Clear caches and run garbage collection before next N
        jax.clear_caches()
        gc.collect()

    print("-" * 65)
    return N_values, times_per_step

resolutions, times = benchmark_scalability()

In [ ]:
import matplotlib.pyplot as plt

# Apply a clean, modern aesthetic
plt.style.use('seaborn-v0_8-darkgrid')

def plot_scalability(N_values, times_per_step):
    fig, ax = plt.subplots(figsize=(8, 5), dpi=150)

    # Calculate total grid points for the x-axis mapping
    grid_points = [6 * n**2 for n in N_values]

    ax.plot(grid_points, times_per_step, marker='o', markersize=8, linestyle='-',
            linewidth=2, color='#2c3e50', label='JAX (GPU) lax.scan')

    ax.set_title('Cubed-Sphere SWE Solver: Scalability on GPU', fontsize=14, pad=15, fontweight='bold')
    ax.set_xlabel('Total Grid Points ($6 \times N^2$)', fontsize=12)
    ax.set_ylabel('Wall-clock Time per Step (ms)', fontsize=12)

    # Secondary X-axis for showing 'N' values directly
    secax = ax.secondary_xaxis('top', functions=(lambda x: (x/6)**0.5, lambda x: 6*x**2))
    secax.set_xlabel('Cubed-Sphere Resolution ($N$)', fontsize=11, labelpad=10)
    secax.set_xticks(N_values)

    ax.grid(True, alpha=0.6, linestyle='--')
    ax.legend(loc='upper left', fontsize=11)

    # Save for showcase/portfolio
    plt.tight_layout()
    plt.savefig('swe_scalability_benchmark.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_scalability(resolutions, times)